In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from keras.utils import normalize
from keras.models import Sequential
from keras.layers import Dense
from keras.metrics import BinaryAccuracy, AUC
from keras.regularizers import l1_l2

from tensorflow.keras.callbacks import EarlyStopping

2022-10-21 12:23:46.377368: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/Immobilienrechner-Challenge/data/main/clean.csv")
data = data[['price', 'zip_code', 'living_space', 'rooms', 'type']]
plz  = pd.read_csv("https://raw.githubusercontent.com/Immobilienrechner-Challenge/data/main/plz_data.csv") 

In [3]:
data["type"].unique()

array(['penthouse', 'terrace-house', 'detached-house', 'flat',
       'stepped-house', 'farmhouse', 'semi-detached-house',
       'stepped-apartment', 'duplex-maisonette', 'attic-flat', 'loft',
       'chalet', 'villa', 'attic-room', 'secondary-suite', 'castle',
       'detached-secondary-suite', 'studio',
       'furnished-residential-property', 'rustico', 'single-room'],
      dtype=object)

In [4]:
data = pd.get_dummies(data, columns=["type"])
data = data.merge(plz, how = 'inner', left_on = "zip_code", right_on = "PLZ")
data = data.drop(columns = "PLZ")

print(len(data))
data

13068


,price,zip_code,living_space,rooms,type_attic-flat,type_attic-room,type_castle,type_chalet,type_detached-house,type_detached-secondary-suite,...,PLZ_Group_9000,PLZ_Group_9100,PLZ_Group_9200,PLZ_Group_9300,PLZ_Group_9400,PLZ_Group_9500,PLZ_Group_9600,PLZ_Group_9700,PLZ_Group_9800,PLZ_Group_9900
0,1150000.0,5023,100.0,3.5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1420000.0,5023,156.0,4.5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1430000.0,5023,154.0,4.5,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,550000.0,5023,124.0,4.5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,720000.0,5022,93.0,2.5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13063,895000.0,8405,130.0,4.5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13064,1090000.0,8405,147.0,5.5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13065,2750000.0,8405,385.0,7.5,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
13066,1415000.0,8472,150.0,6.5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
data.isna().sum().sort_values(ascending = False)

price                                1011
living_space                          692
rooms                                 582
PLZ_Group_5000                          0
PLZ_Group_4200                          0
                                     ... 
gde_politics_fdp                        0
gde_social_help_quota                   0
gde_area_nonproductive_percentage       0
gde_foreigners_percentage               0
PLZ_Group_9900                          0
Length: 163, dtype: int64

In [6]:
data = data.dropna()
len(data)

11263

In [7]:
y_cols = data.columns[data.columns.str.startswith("type_")]
X_cols = list(set(data._get_numeric_data().columns) - set(y_cols))

X = data[X_cols]
y = data[y_cols]

In [8]:
X = normalize(X, axis=-1, order=2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
n_epochs = 500
batchsize = 256

n_inputs = X_train.shape[1]
n_outputs = y_train.shape[1]

n_hiddenlayers = 8
n_nodes_per_layer = 128

model = Sequential()

model.add(
        Dense(n_nodes_per_layer, activation='relu', input_shape=(n_inputs,))
    )

for i in range(n_hiddenlayers):
    model.add(
        Dense(n_nodes_per_layer, activation='relu')
    )

model.add(
        Dense(n_outputs, activation='softmax')
)

model.compile(
        optimizer='adam', 
        loss='binary_crossentropy', 
        metrics=['accuracy', 'AUC']
)

early_stop = EarlyStopping(
        monitor='val_loss', 
        mode='min', 
        verbose=1, 
        patience=50
)

history = model.fit(
        X_train, 
        y_train, 
        epochs=n_epochs, 
        batch_size=batchsize, 
        validation_data=(X_test, y_test),
        callbacks=[early_stop],
        use_multiprocessing=True
)

Epoch 1/500
2022-10-21 12:23:47.905798: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: UNKNOWN ERROR (34)
2022-10-21 12:23:47.905835: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-985da17b-3a43-4cd2-aed4-b3aeabe863c4): /proc/driver/nvidia/version does not exist
2022-10-21 12:23:47.906037: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
36/36 [==============================] - 1s 17ms/step - loss: 0.3988 - accuracy: 0.1604 - auc: 0.7124 - val_loss: 0.1301 - val_accuracy: 0.4643 - val_auc: 0.8954
Epoch 2/500
36/36 [==============================] - 0s 11ms/step - loss: 0.1247 - accuracy: 0.4597 - auc: 0.90

In [10]:
for i, val in enumerate(model.predict(X_test)[5]):
    print(f'{y.columns[i]}: {val:.2f}')

71/71 [==============================] - 0s 1ms/step
type_attic-flat: 0.01
type_attic-room: 0.00
type_castle: 0.00
type_chalet: 0.01
type_detached-house: 0.42
type_detached-secondary-suite: 0.00
type_duplex-maisonette: 0.03
type_farmhouse: 0.01
type_flat: 0.27
type_furnished-residential-property: 0.00
type_loft: 0.00
type_penthouse: 0.05
type_rustico: 0.00
type_secondary-suite: 0.00
type_semi-detached-house: 0.08
type_single-room: 0.00
type_stepped-apartment: 0.03
type_stepped-house: 0.01
type_studio: 0.00
type_terrace-house: 0.06
type_villa: 0.02


In [11]:
for i, val in enumerate(y_test.values[5]):
    print(f'{y.columns[i]}: {val:.0f}')

type_attic-flat: 0
type_attic-room: 0
type_castle: 0
type_chalet: 0
type_detached-house: 1
type_detached-secondary-suite: 0
type_duplex-maisonette: 0
type_farmhouse: 0
type_flat: 0
type_furnished-residential-property: 0
type_loft: 0
type_penthouse: 0
type_rustico: 0
type_secondary-suite: 0
type_semi-detached-house: 0
type_single-room: 0
type_stepped-apartment: 0
type_stepped-house: 0
type_studio: 0
type_terrace-house: 0
type_villa: 0


In [12]:
model.evaluate(X_test, y_test)

71/71 [==============================] - 0s 2ms/step - loss: 0.1120 - accuracy: 0.5308 - auc: 0.9271


[0.11201939731836319, 0.5308477878570557, 0.9271179437637329]

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=985da17b-3a43-4cd2-aed4-b3aeabe863c4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>